In [102]:
from dotenv import load_dotenv
import os
import base64
import json
from requests import post,get

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

### Get Token

In [2]:
def get_token():
    auth_string = client_id+":"+client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes),"utf-8")

    url = "https://accounts.spotify.com/api/token"

    headers = {
        "Authorization": "Basic "+ auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}

    result = post(url, headers = headers, data = data) # send a post request to the server, return the 
    json_result = json.loads(result.content)
    token = json_result["access_token"]

    return token

def get_auth_header(token):
    return {"Authorization":"Bearer "+token}


### Search for artist

In [58]:
# list out 5 artists and select one.
# input: artist name (str)
# output: return a dict contain{"name","url","image_url","id"}


def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=5" # artist name, type*, limit 

    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)["artists"]["items"]

    # Check if any result
    if len(json_result) == 0:
        print("No artist with this name exists...")
        return None
    
    # list out 5 results and select one 
    print("The results of searching artist: ")
    for i,artist in enumerate(json_result):
        # popularity = artist["popularity"]
        name = artist["name"]
        print(f"{i+1}. {name}")

    idx = int(input("Select an artist"))-1 # input
    print("------------------------------")
    print(f'Your selection is: {json_result[idx]["name"]} \n')

    
    artist = {
        "name" : json_result[idx]["name"],
        "genres" : json_result[idx]["genres"],
        "url" : json_result[idx]["external_urls"]["spotify"],
        "image_url" : json_result[idx]["images"][1]["url"],
        "id" : json_result[idx]["id"]
    }
    
    return artist

### Search for track

In [64]:
# list out 5 tracks
# artist:{"name","artists","popularity","id","external_urls"}

def search_for_track(token, track_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={track_name}&type=track&limit=5" # track name, type*, limit 

    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)["tracks"]["items"]

    # Check if any result
    if len(json_result) == 0:
        print("No track with this name exists...")
        return None
    
    # list out 5 results and select one 
    print("The results of searching track: ")
    for i,track in enumerate(json_result):
        name = track["name"]
        artists = ", ".join([ artist["name"] for artist in track["artists"]])
        print(f"{i+1}. {name} - {artists}")

    idx = int(input("Select a track"))-1 # input
    artists = json_result[idx]["artists"]
    
    artists_name = "、".join([artist["name"] for artist in artists])
    track_name = json_result[idx]["name"]
    
    # form a dict storing the seletced track's info.
    track = {
        "name" :  f"{track_name} - {artists_name}",
        "url" : json_result[idx]["external_urls"]["spotify"],
        "id" : json_result[idx]["id"]
    }

    print("------------------------------")
    print(f'Your selection is: {track["name"]} \n')
    
    return track

In [93]:
# 需要搭配 autocomplete 之類的

def get_genres(token):
    url = "https://api.spotify.com/v1/recommendations/available-genre-seeds"
    headers = get_auth_header(token)
    result = get(url, headers = headers)
    json_result = json.loads(result.content)["genres"]

    # search = input("Search for genres")
    # result = []
    # for i in json_result:
    #     if search in i:
    #         result.append(i)
    # idx = input("Select a genre")
    # genre = result[idx-1]
    # return genre
    
    return json_result

### Get recommendations

In [100]:
def get_recommedations(token, selections,limit=5 ):

    # Check selections, then create query
    query = []
    for seed in ['artists', 'tracks', 'genres']:
        if len(selections[seed])>0:
            id = ",".join([i["id"] for i in selections[seed]])
            if seed != "genres":
                query.append(f"seed_{seed}={id}")
            else:
                query.append(f"{seed}={id}")

    querys = "&".join(query)

    # send a GET request to endpoint
    url = "https://api.spotify.com/v1/recommendations?"+querys+f"&limit={limit}"
    headers = get_auth_header(token)
    result = get(url, headers = headers)
    json_result = json.loads(result.content)["tracks"]

    
    # print out the selectoin(for user check)
    for seed in ['artists', 'tracks', 'genres']:
        if len(selections[seed])>0:
            print(f"{seed[0].upper()+seed[1:]}:")
            for idx,i in enumerate(selections[seed]):
                print(f'{idx+1}. {i["name"]}')

    print("------------------------------")
    
    # print out recommedations
    print("Your Recommendation:")
    for i,recom in enumerate(json_result):
        track = recom["name"]
        artists = ", ".join([artist["name"] for artist in recom["artists"]])

        print(f"{i+1}. {track} - {artists}")

    return json_result

### Main script

In [95]:
token = get_token()
selections = {"artists":[],"tracks":[],"genres":[]}

In [96]:
artist = search_for_artist(token, "vaundy")
selections["artists"].append(artist)

track = search_for_track(token, "yorunikakeru")
selections["tracks"].append(track)

The results of searching artist: 
1. Vaundy
2. Vantage
3. Fujii Kaze
4. Valntn
5. Kenshi Yonezu
------------------------------
Your selection is: Vaundy 

The results of searching track: 
1. Yorunikakeru (Two Pianos Four Hands) - Jacob Koller, よみぃ
2. 夜に駆ける - YOASOBI
3. Yoruwahonoka - Eve
4. 夜に駆ける (カバー) - オサム
5. Yoruwahonoka - Eve
------------------------------
Your selection is: 夜に駆ける - YOASOBI 



In [101]:
recoms = get_recommedations(token=token, selections = selections)

Artists:
1. Vaundy
Tracks:
1. 夜に駆ける - YOASOBI
------------------------------
Your Recommendation:
1. 檸檬の日々 - WurtS
2. 六兆年と一夜物語 - Kano
3. Ginger - TOMOO
4. 銃の部品 - PEOPLE 1
5. Omoideshiritori - DIALOGUE+
